# Test notebook

## Library installation and modules imports
We uploaded on PyPi our own library containing the functions needed to define the models and preprocess the images, the ratio for this is to allow for a leaner structure and better redability of the notebook

In [ ]:
!pip install Human_Data_Analytics

In [16]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
import os
import time

# our models uploaded with pypi as a python package
from HDA.preprocessing.image_pre_processing import data_generator_patch

## Inception-v4

In [17]:
num_patches_per_side = 5
crop_list={'top': 0.1, 'bottom':0.05, 'left':0.1, 'right':0.1}
test_path = "/kaggle/input/dataset-boneage/test"
inception_model_path = "/kaggle/input/boneageprediction/tensorflow2/default/35/Inceptionv4.h5"

In [18]:
# InceptionV4/ResNet18/ResNet18 with channel attention
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    model = load_model(inception_model_path, compile=False)

In [19]:
extracted_files = os.listdir(os.path.join(test_path, 'images'))
num_test_images=len(extracted_files)
print("Number of validation images:", num_test_images)

labels_test = pd.read_csv(os.path.join(test_path, "labels.csv"))
labels_test_age = labels_test['Bone Age (months)'].to_list()
gender_test = labels_test['male'].apply(lambda x: 1 if x == True else 0).to_list()

preprocessed_images_test = tf.data.Dataset.from_generator(
    lambda: data_generator_patch(os.path.join(test_path, "images"),
                           gender_test,
                           labels_test_age, train = False, 
                           num_patches_per_side=num_patches_per_side,
                           crop_list=crop_list),
        output_signature=(
        (tf.TensorSpec(shape=(224, 224, 3), dtype=tf.float32, name="image"),  # Input shape
        tf.TensorSpec(shape=(), dtype=tf.float32, name="gender")),   # Gender shape
        tf.TensorSpec(shape=(), dtype=tf.float32, name="label")   # Target shape
    )
)

# Optionally, you can shuffle and prefetch for performance
batch_size = 40
preprocessed_images_test = preprocessed_images_test.repeat()
preprocessed_images_test = preprocessed_images_test.batch(batch_size)
test_dataset = preprocessed_images_test.prefetch(2)
test_patch_num = num_patches_per_side**2
test_steps = int(np.ceil(len(labels_test_age)/(batch_size/test_patch_num)))

Number of validation images: 200


In [20]:
start = time.time()
test_values = model.predict(test_dataset, steps=test_steps).squeeze() # , steps=test_steps

median_predictions = [np.median(test_values[i:i+test_patch_num]) for i in range(0, len(test_values), test_patch_num)]
median_loss = tf.keras.losses.MAE(
    labels_test_age, median_predictions
)
end = time.time()

print("Median of the patches: ",  float(median_loss))
print("Time for the full test computation: ", end - start)

125/125 ━━━━━━━━━━━━━━━━━━━━ 25s 124ms/step
Median of the patches:  6.338632106781006
Time for the full test computation:  25.459535360336304


## ResNet18 with Channel Attention

In [21]:
num_patches_per_side = 5
crop_list={'top': 0.05, 'bottom':0.05, 'left':0.05, 'right':0.05}
test_path = "/kaggle/input/dataset-boneage/test"
resnet_model_path = "/kaggle/input/boneageprediction/tensorflow2/default/35/ResNet18_channel_attention.h5"

In [22]:
# InceptionV4/ResNet18/ResNet18 with channel attention
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    model = load_model(resnet_model_path, compile=False)

In [23]:
extracted_files = os.listdir(os.path.join(test_path, 'images'))
num_test_images=len(extracted_files)
print("Number of validation images:", num_test_images)

labels_test = pd.read_csv(os.path.join(test_path, "labels.csv"))
labels_test_age = labels_test['Bone Age (months)'].to_list()
gender_test = labels_test['male'].apply(lambda x: 1 if x == True else 0).to_list()

preprocessed_images_test = tf.data.Dataset.from_generator(
    lambda: data_generator_patch(os.path.join(test_path, "images"),
                           gender_test,
                           labels_test_age, train = False, 
                           num_patches_per_side=num_patches_per_side,
                           crop_list=crop_list),
        output_signature=(
        (tf.TensorSpec(shape=(224, 224, 3), dtype=tf.float32, name="image"),  # Input shape
        tf.TensorSpec(shape=(), dtype=tf.float32, name="gender")),   # Gender shape
        tf.TensorSpec(shape=(), dtype=tf.float32, name="label")   # Target shape
    )
)

# Optionally, you can shuffle and prefetch for performance
batch_size = 40
preprocessed_images_test = preprocessed_images_test.repeat()
preprocessed_images_test = preprocessed_images_test.batch(batch_size)
test_dataset = preprocessed_images_test.prefetch(2)
test_patch_num = num_patches_per_side**2
test_steps = int(np.ceil(len(labels_test_age)/(batch_size/test_patch_num)))

Number of validation images: 200


In [24]:
start = time.time()
test_values = model.predict(test_dataset, steps=test_steps).squeeze() # , steps=test_steps

median_predictions = [np.median(test_values[i:i+test_patch_num]) for i in range(0, len(test_values), test_patch_num)]
median_loss = tf.keras.losses.MAE(
    labels_test_age, median_predictions
)
end = time.time()

print("Median of the patches: ",  float(median_loss))
print("Time for the full test computation: ", end - start)

125/125 ━━━━━━━━━━━━━━━━━━━━ 13s 89ms/step
Median of the patches:  7.614485263824463
Time for the full test computation:  13.548022508621216
